In [93]:
import platform
import numpy as np
import pandas as pd
import random

import torch 
from torch import optim 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

from tqdm.notebook import tqdm
from transformers import AutoTokenizer

# enable tqdm in pandas
tqdm.pandas()

# select device
if torch.cuda.is_available():
    device = torch.device('cuda')
elif 'arm64' in platform.platform():
    device = torch.device('mps') # 'mps'
else:
    device = torch.device('cpu')
print(f'device: {device.type}') 

# random seed
seed = 1234

# pytorch ignores this label in the loss
ignore_index = -100

# set random seed
if seed is not None:
    print(f'random seed: {seed}')
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

# which transformer to use
transformer_name = "bert-base-cased" # 'xlm-roberta-base' # 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(transformer_name)

device: mps
random seed: 1234


In [94]:

# map labels to the first token in each word
def align_labels(word_ids, labels, label_to_index):
    label_ids = []
    previous_word_id = None
    for word_id in word_ids:
        if word_id is None or word_id == previous_word_id:
            # ignore if not a word or word id has already been seen
            label_ids.append(ignore_index)
        else:
            # get label id for corresponding word
            label_id = label_to_index[labels[word_id]]
            label_ids.append(label_id)
        # remember this word id
        previous_word_id = word_id
    
    return label_ids
            
# build a set of labels in the dataset            
def read_label_set(fn):
    labels = set()
    with open(fn) as f:
        for index, line in enumerate(f):
            line = line.strip()
            tokens = line.split()
            if tokens != []:
                label = tokens[-1]
                labels.add(label)
    return labels

# converts a two-column file in the basic MTL format ("word \t label") into a dataframe
def read_dataframe(fn, label_to_index, task_id):
    # now build the actual dataframe for this dataset
    data = {'words': [], 'str_labels': [], 'input_ids': [], 'word_ids': [], 'labels': [], 'task_ids': []}
    with open(fn) as f:
        sent_words = []
        sent_labels = [] 
        for index, line in tqdm(enumerate(f)):
            line = line.strip()
            tokens = line.split()
            if tokens == []:
                data['words'].append(sent_words)
                data['str_labels'].append(sent_labels)
                
                # tokenize each sentence
                token_input = tokenizer(sent_words, is_split_into_words = True)  
                token_ids = token_input['input_ids']
                word_ids = token_input.word_ids(batch_index = 0)
                
                # map labels to the first token in each word
                token_labels = align_labels(word_ids, sent_labels, label_to_index)
                
                data['input_ids'].append(token_ids)
                data['word_ids'].append(word_ids)
                data['labels'].append(token_labels)
                data['task_ids'].append(task_id)
                sent_words = []
                sent_labels = [] 
            else:
                sent_words.append(tokens[0])
                sent_labels.append(tokens[1])
    return pd.DataFrame(data)


In [95]:
class Task():
    def __init__(self, task_id, train_file_name, dev_file_name, test_file_name):
        self.task_id = task_id
        # we need an index of labels first
        self.labels = read_label_set(train_file_name)
        self.index_to_label = {i:t for i,t in enumerate(self.labels)}
        self.label_to_index = {t:i for i,t in enumerate(self.labels)}
        self.num_labels = len(self.index_to_label)
        # create data frames for the datasets
        self.train_df = read_dataframe(train_file_name, self.label_to_index, self.task_id)
        self.dev_df = read_dataframe(dev_file_name, self.label_to_index, self.task_id)
        self.test_df = read_dataframe(test_file_name, self.label_to_index, self.task_id)
                

In [96]:
ner_task = Task(0, "data/conll-ner/train.txt", "data/conll-ner/dev.txt", "data/conll-ner/test.txt")
pos_task = Task(1, "data/pos/train.txt", "data/pos/dev.txt", "data/pos/test.txt")

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [97]:
ner_task.train_df

,words,str_labels,input_ids,word_ids,labels,task_ids
0,[-DOCSTART-],[O],"[101, 118, 141, 9244, 9272, 12426, 1942, 118, ...","[None, 0, 0, 0, 0, 0, 0, 0, None]","[-100, 2, -100, -100, -100, -100, -100, -100, ...",0
1,"[EU, rejects, German, call, to, boycott, Briti...","[B-ORG, O, B-MISC, O, O, O, B-MISC, O, O]","[101, 7270, 22961, 1528, 1840, 1106, 21423, 14...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]","[-100, 6, 2, 4, 2, 2, 2, 4, 2, -100, 2, -100]",0
2,"[Peter, Blackburn]","[B-PER, I-PER]","[101, 1943, 14428, 102]","[None, 0, 1, None]","[-100, 0, 7, -100]",0
3,"[BRUSSELS, 1996-08-22]","[B-LOC, O]","[101, 26660, 13329, 12649, 15928, 1820, 118, 4...","[None, 0, 0, 0, 0, 1, 1, 1, 1, 1, None]","[-100, 1, -100, -100, -100, 2, -100, -100, -10...",0
4,"[The, European, Commission, said, on, Thursday...","[O, B-ORG, I-ORG, O, O, O, O, O, O, B-MISC, O,...","[101, 1109, 1735, 2827, 1163, 1113, 9170, 1122...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 1...","[-100, 2, 6, 3, 2, 2, 2, 2, 2, 2, 4, 2, 2, 2, ...",0
...,...,...,...,...,...,...
14982,"[on, Friday, :]","[O, O, O]","[101, 1113, 5286, 131, 102]","[None, 0, 1, 2, None]","[-100, 2, 2, 2, -100]",0
14983,"[Division, two]","[O, O]","[101, 1784, 1160, 102]","[None, 0, 1, None]","[-100, 2, 2, -100]",0
14984,"[Plymouth, 2, Preston, 1]","[B-ORG, O, B-ORG, O]","[101, 10033, 123, 8083, 122, 102]","[None, 0, 1, 2, 3, None]","[-100, 6, 2, 6, 2, -100]",0
14985,"[Division, three]","[O, O]","[101, 1784, 1210, 102]","[None, 0, 1, None]","[-100, 2, 2, -100]",0


In [98]:
pos_task.train_df

,words,str_labels,input_ids,word_ids,labels,task_ids
0,"[Pierre, Vinken, ,, 61, years, old, ,, will, j...","[NNP, NNP, ,, CD, NNS, JJ, ,, MD, VB, DT, NN, ...","[101, 4855, 25354, 6378, 117, 5391, 1201, 1385...","[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11...","[-100, 2, 2, -100, 9, 29, 23, 5, 9, 25, 30, 34...",1
1,"[Mr., Vinken, is, chairman, of, Elsevier, N.V....","[NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, NNP, ...","[101, 1828, 119, 25354, 6378, 1110, 3931, 1104...","[None, 0, 0, 1, 1, 2, 3, 4, 5, 5, 5, 6, 6, 6, ...","[-100, 2, -100, 2, -100, 24, 3, 11, 2, -100, -...",1
2,"[Rudolph, Agnew, ,, 55, years, old, and, forme...","[NNP, NNP, ,, CD, NNS, JJ, CC, JJ, NN, IN, NNP...","[101, 19922, 138, 8376, 2246, 117, 3731, 1201,...","[None, 0, 1, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10,...","[-100, 2, 2, -100, -100, 9, 29, 23, 5, 21, 5, ...",1
3,"[A, form, of, asbestos, once, used, to, make, ...","[DT, NN, IN, NN, RB, VBN, TO, VB, NNP, NN, NNS...","[101, 138, 1532, 1104, 1112, 12866, 11990, 151...","[None, 0, 1, 2, 3, 3, 3, 4, 5, 6, 7, 8, 9, 10,...","[-100, 34, 3, 11, 3, -100, -100, 26, 4, 31, 30...",1
4,"[The, asbestos, fiber, ,, crocidolite, ,, is, ...","[DT, NN, NN, ,, NN, ,, VBZ, RB, JJ, IN, PRP, V...","[101, 1109, 1112, 12866, 11990, 12753, 117, 17...","[None, 0, 1, 1, 1, 2, 3, 4, 4, 4, 4, 4, 5, 6, ...","[-100, 34, 3, -100, -100, 3, 9, 3, -100, -100,...",1
...,...,...,...,...,...,...
46787,"[Says, Peter, Mokaba, ,, president, of, the, S...","[VBZ, NNP, NNP, ,, NN, IN, DT, NNP, NNP, NNP, ...","[101, 8652, 1116, 1943, 12556, 1968, 2822, 117...","[None, 0, 0, 1, 2, 2, 2, 3, 4, 5, 6, 7, 8, 9, ...","[-100, 24, -100, 2, 2, -100, -100, 9, 3, 11, 3...",1
46788,"[They, never, considered, themselves, to, be, ...","[PRP, RB, VBD, PRP, TO, VB, NN, RB, .]","[101, 1220, 1309, 1737, 2310, 1106, 1129, 1625...","[None, 0, 1, 2, 3, 4, 5, 6, 7, 8, None]","[-100, 27, 26, 44, 27, 31, 30, 3, 26, 15, -100]",1
46789,"[At, last, night, 's, rally, ,, they, called, ...","[IN, JJ, NN, POS, NN, ,, PRP, VBD, IN, PRP$, N...","[101, 1335, 1314, 1480, 112, 188, 11158, 117, ...","[None, 0, 1, 2, 3, 3, 4, 5, 6, 7, 8, 9, 10, 11...","[-100, 11, 5, 3, 19, -100, 3, 9, 27, 44, 11, 4...",1
46790,"[``, We, emphasize, discipline, because, we, k...","[``, PRP, VBP, NN, IN, PRP, VBP, IN, DT, NN, V...","[101, 169, 169, 1284, 19291, 9360, 1272, 1195,...","[None, 0, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11...","[-100, 16, -100, 27, 32, 3, 11, 27, 32, 11, 34...",1


In [99]:
from transformers.modeling_outputs import TokenClassifierOutput
from transformers import PreTrainedModel
from transformers import AutoConfig, AutoModel

# This class is adapted from: https://towardsdatascience.com/how-to-create-and-train-a-multi-task-transformer-model-18c54a146240
class TokenClassificationModel(PreTrainedModel):    
    def __init__(self, config, tasks):
        super().__init__(config)
        self.encoder = AutoModel.from_pretrained(transformer_name, config=config)

        self.output_heads = nn.ModuleDict()
        for task in tasks:
            head = TokenClassificationHead(self.encoder.config.hidden_size, task.num_labels, config.hidden_dropout_prob)
            # ModuleDict requires keys to be strings
            self.output_heads[str(task.task_id)] = head
        
        self._init_weights()
        
    def _init_weights(self):
        for task_id in self.output_heads:
            self.output_heads[task_id]._init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, labels=None, task_ids=None, **kwargs):
        outputs = self.encoder(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            **kwargs,
        )
        sequence_output = outputs[0]
        
        #print(f'batch size = {len(input_ids)}')
        #print(f'task_ids in this batch: {task_ids}')
        
        # generate specific predictions and losses for each task head
        unique_task_ids_list = torch.unique(task_ids).tolist()
        logits = None
        loss_list = []
        for unique_task_id in unique_task_ids_list:
            task_id_filter = task_ids == unique_task_id
            filtered_sequence_output = sequence_output[task_id_filter]
            filtered_labels = None if labels is None else labels[task_id_filter]
            filtered_attention_mask = None if attention_mask is None else attention_mask[task_id_filter]
            #print(f'size of batch for task {unique_task_id} is: {len(filtered_sequence_output)}')
            logits, task_loss = self.output_heads[str(unique_task_id)].forward(
                filtered_sequence_output, None,
                filtered_labels,
                filtered_attention_mask,
            )
            if filtered_labels is not None:
                loss_list.append(task_loss)
                
        loss = None if len(loss_list) == 0 else torch.stack(loss_list)
                    
        # logits are only used for eval, in which case we handle a single task at a time
        # TODO: allow all tasks in the forward pass at inference                     
        return TokenClassifierOutput(
            loss=loss.mean(),
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

class TokenClassificationHead(nn.Module):
    def __init__(self, hidden_size, num_labels, dropout_p=0.1):
        super().__init__()
        self.dropout = nn.Dropout(dropout_p)
        self.classifier = nn.Linear(hidden_size, num_labels)
        self.num_labels = num_labels

        self._init_weights()

    def _init_weights(self):
        self.classifier.weight.data.normal_(mean=0.0, std=0.02)
        if self.classifier.bias is not None:
            self.classifier.bias.data.zero_()

    def forward(self, sequence_output, pooled_output, labels=None, attention_mask=None, **kwargs):
        sequence_output_dropout = self.dropout(sequence_output)
        logits = self.classifier(sequence_output_dropout)
        
        loss = None
        if labels is not None:
            loss_fn = nn.CrossEntropyLoss()            
            inputs = logits.view(-1, self.num_labels)
            targets = labels.view(-1)
            loss = loss_fn(inputs, targets)

        return logits, loss

In [100]:
tasks = [ner_task, pos_task]
config = AutoConfig.from_pretrained(transformer_name)
model = TokenClassificationModel(config, tasks)


In [101]:
from transformers import TrainingArguments

num_epochs = 1
batch_size = 128
weight_decay = 0.01
model_name = f'{transformer_name}-ner'

use_mps_device = True if str(device) == 'mps' else False

training_args = TrainingArguments(
    output_dir=model_name,
    log_level='error',
    num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    # evaluation_strategy='epoch',
    do_eval=False, # we will evaluate each task explicitly
    weight_decay=weight_decay,
    use_mps_device = use_mps_device
)

training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=True,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=no,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
label_name

In [102]:
from sklearn.metrics import accuracy_score

def compute_metrics(eval_pred):
    # gold labels
    label_ids = eval_pred.label_ids
    # predictions
    pred_ids = np.argmax(eval_pred.predictions, axis=-1)
    # collect gold and predicted labels, ignoring ignore_index label
    y_true, y_pred = [], []
    batch_size, seq_len = pred_ids.shape
    for i in range(batch_size):
        for j in range(seq_len):
            if label_ids[i, j] != ignore_index:
                y_true.append(label_ids[i][j]) #index_to_label[label_ids[i][j]])
                y_pred.append(pred_ids[i][j]) #index_to_label[pred_ids[i][j]])
    # return computed metrics
    return {'accuracy': accuracy_score(y_true, y_pred)}

In [103]:
from datasets import Dataset, DatasetDict

ds = DatasetDict()
ds['train'] = Dataset.from_pandas(pd.concat([ner_task.train_df, pos_task.train_df]))
ds['validation'] = Dataset.from_pandas(pd.concat([ner_task.dev_df, pos_task.dev_df]))
ds['test'] = Dataset.from_pandas(pd.concat([ner_task.test_df, pos_task.test_df]))

# these are no longer needed; discard them to save memory
ner_task.train_df = None
pos_task.train_df = None

ds

DatasetDict({
    train: Dataset({
        features: ['words', 'str_labels', 'input_ids', 'word_ids', 'labels', 'task_ids', '__index_level_0__'],
        num_rows: 61779
    })
    validation: Dataset({
        features: ['words', 'str_labels', 'input_ids', 'word_ids', 'labels', 'task_ids', '__index_level_0__'],
        num_rows: 8504
    })
    test: Dataset({
        features: ['words', 'str_labels', 'input_ids', 'word_ids', 'labels', 'task_ids', '__index_level_0__'],
        num_rows: 6101
    })
})

In [104]:
def evaluate(trainer, task, name):
    print(f"Evaluating on the validation dataset for task {name}:")
    ds = Dataset.from_pandas(task.dev_df)
    scores = trainer.evaluate(ds)
    return scores

In [105]:
from transformers import Trainer
from transformers import DataCollatorForTokenClassification
from datetime import datetime

data_collator = DataCollatorForTokenClassification(tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    train_dataset=ds['train'],
    eval_dataset=ds['validation'],
    tokenizer=tokenizer,
)

now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

trainer.train()

now = datetime.now()
print("Current Time =", now.strftime("%H:%M:%S"))

evaluate(trainer, ner_task, "NER")
evaluate(trainer, pos_task, "POS")

Current Time = 15:21:21


/Users/msurdeanu/miniconda3/envs/transformers-mps/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


{'train_runtime': 2078.8621, 'train_samples_per_second': 29.718, 'train_steps_per_second': 0.232, 'train_loss': 0.051111513536662545, 'epoch': 1.0}
Current Time = 15:56:00
Evaluating on the validation dataset for task NER:
{'eval_loss': 0.010295262560248375, 'eval_accuracy': 0.9876885493815193, 'eval_runtime': 13.6361, 'eval_samples_per_second': 254.178, 'eval_steps_per_second': 2.053, 'epoch': 1.0}
Evaluating on the validation dataset for task POS:
{'eval_loss': 0.021720489487051964, 'eval_accuracy': 0.9798711154238511, 'eval_runtime': 22.5231, 'eval_samples_per_second': 223.681, 'eval_steps_per_second': 1.776, 'epoch': 1.0}


{'eval_loss': 0.021720489487051964,
 'eval_accuracy': 0.9798711154238511,
 'eval_runtime': 22.5231,
 'eval_samples_per_second': 223.681,
 'eval_steps_per_second': 1.776,
 'epoch': 1.0}

In [106]:
from sklearn.metrics import classification_report

def classification_report_on_test(trainer, task, name):
    print(f"Test classification report for task {name}:")
    num_labels = task.num_labels
    ds = Dataset.from_pandas(task.test_df)
    output = trainer.predict(ds)
    label_ids = output.label_ids.reshape(-1)
    predictions = output.predictions.reshape(-1, num_labels)
    predictions = np.argmax(predictions, axis=-1)
    mask = label_ids != ignore_index
    
    y_true = label_ids[mask]
    y_pred = predictions[mask]
    target_names = [task.index_to_label.get(ele, "") for ele in range(num_labels)]
    print(target_names)
    
    report = classification_report(
        y_true, y_pred,
        target_names=target_names
    )
    print(report)
    print()


In [107]:
classification_report_on_test(trainer, ner_task, "NER")
classification_report_on_test(trainer, pos_task, "POS")

Test classification report for task NER:
['B-PER', 'B-LOC', 'O', 'I-ORG', 'B-MISC', 'I-MISC', 'B-ORG', 'I-PER', 'I-LOC']
              precision    recall  f1-score   support

       B-PER       0.96      0.95      0.96      1617
       B-LOC       0.92      0.92      0.92      1668
           O       1.00      0.99      0.99     38554
       I-ORG       0.86      0.89      0.87       835
      B-MISC       0.76      0.82      0.79       702
      I-MISC       0.63      0.73      0.68       216
       B-ORG       0.89      0.88      0.89      1661
       I-PER       0.98      0.99      0.99      1156
       I-LOC       0.80      0.87      0.83       257

    accuracy                           0.98     46666
   macro avg       0.87      0.89      0.88     46666
weighted avg       0.98      0.98      0.98     46666


Test classification report for task POS:
['FW', 'SYM', 'NNP', 'NN', 'VBN', 'JJ', ':', "''", '#', ',', 'VBG', 'IN', 'RP', 'PDT', 'NNPS', '.', '``', 'RBS', '-RRB-', 'POS', '-L

/Users/msurdeanu/miniconda3/envs/transformers-mps/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/msurdeanu/miniconda3/envs/transformers-mps/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/msurdeanu/miniconda3/envs/transformers-mps/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa